# Razonamiento de los Agentes
Este código muestra cómo implementar razonamiento en agentes que interactúan con el entorno.

## 1.- Setup inicial

### 1.1- Instalar librerías

In [ ]:
#! pip install -r requirements.txt

### 1.2.- Cargar librerías

In [ ]:
import openai
import os
from dotenv import load_dotenv
from llm import generate_text, gpt4_model
from agents.memory import LastMemories

### 1.3.- Variables de entorno

In [ ]:
# Load secrets and config from .env file
load_dotenv()

# OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Model endpoint names
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt35_model))

## 2.- Tecnicas de Razonamiento

### 2.1 - Ejemplos sin razonamiento 

In [ ]:
user_input = f"Toma la última letra de cada palabra en 'Larry Page' y concaténalas."
response = generate_text(user_input)
print(response) #Resp: ye

In [ ]:
user_input = f"Tom dejo 3 calcetines en la lavadora, al finalizar, dejo los 3 calcetines secar, luego de 10 horas los calcetines se secaron completamente. ¿Cuánto tiempo llevara secar 20 calcetines?."
response = generate_text(user_input)
print(response) #Resp: 10 Horas

In [ ]:
user_input = f"Jane nació el último día de febrero de 2001. Hoy es su cumpleaños de 16 años. Cual es la fecha ayer en DD/MM/YYYY?"
response = generate_text(user_input)
print(response) #Resp: 27/02/2017

### 2.2 - Chain Of Thought (CoT)

In [ ]:
# CoT Prompt
COT_PROMPT = "Piensa con sentido comun, detenida y lógicamente, explicando tu respuesta paso a paso."

user_input = f"Toma la última letra de cada palabra en 'Larry Page' y concaténalas. {COT_PROMPT}"
response = generate_text(user_input)
print(response) #Resp: ye

In [ ]:
user_input = f"Tom dejo 3 calcetines en la lavadora, al finalizar, dejo los 3 calcetines secar, luego de 10 horas los calcetines se secaron completamente. ¿Cuánto tiempo llevara secar 20 calcetines?. {COT_PROMPT}"
response = generate_text(user_input)
print(response) #Resp: 10 Horas

In [ ]:
user_input = f"Jane nació el último día de febrero de 2001. Hoy es su cumpleaños de 16 años. Cual es la fecha ayer en DD/MM/YYYY?. {COT_PROMPT}"
response = generate_text(user_input)
print(response) #Resp: 27/02/2017

### 2.3 - Chain Of Thought (CoT) + Few Shot

In [ ]:
COT_PROMPT = "Piensa con sentido comun, detenida y lógicamente, explicando tu respuesta paso a paso."

# Inyeccion de mensajes de ejemplo
messages = LastMemories()

#Example 1: Concadenate problem
messages.add_to_memory("user","Take the last letters of the words in 'Elon Musk' and concatenate them.")
messages.add_to_memory("assistant","The last letter of 'Elon' is 'n'. The last letter of 'Musk' is 'k'. Concatenating them is 'nk'. The answer is nk.")
#Example 2:  Date Understanding problem
messages.add_to_memory("user","The concert was scheduled to be on 06/01/1943, but was delayed by one day to today. What is the date 10 days ago in MM/DD/YYYY?")
messages.add_to_memory("assistant","One day after 06/01/1943 is 06/02/1943, so today is 06/02/1943. 10 days before today is 05/23/1943. So the answer is 05/23/1943.")
# Example 3: Date Understanding problem
messages.add_to_memory("user","Jane thought today is 3/11/2002, but today is in fact Mar 12, which is 1 day later. What is the date 24 hours later in MM/DD/YYYY?")
messages.add_to_memory("assistant","Today is 03/12/2002. So the date 24 hours later will be 03/13/2002. So the answer is 03/13/2002.")

In [ ]:
user_input = f"Toma la última letra de cada palabra en 'Larry Page' y concaténalas. {COT_PROMPT}"
response = generate_text(user_input, messages=messages.get_last_memories(10))
print(response) #Resp: ye

In [ ]:
user_input = f"Tom dejo 3 calcetines en la lavadora, al finalizar, dejo los 3 calcetines secar, luego de 10 horas los calcetines se secaron completamente. ¿Cuánto tiempo llevara secar 20 calcetines?. {COT_PROMPT}"
response = generate_text(user_input, messages=messages.get_last_memories(10))
print(response) #Resp: 10 Horas

In [ ]:
user_input = f"Jane nació el último día de febrero de 2001. Hoy es su cumpleaños de 16 años. Cual es la fecha ayer en MM/DD/YYYY?. {COT_PROMPT}"
response = generate_text(user_input, messages=messages.get_last_memories(10))
print(response) #Resp: 02/27/2017

### 2.4.- Tree Of Thought (ToT) simplifacado

In [ ]:
N_SOLUTIONS=5
STEPS=3

PROMPT_TOT_GENERATOR = f"""Based on the <QUERY>, generate {N_SOLUTIONS} different logical and common sense experts and consider type of experts in the area of the <QUERY> keep in mind the context, the environment, etc.
Response only one step (one different for each expert), in one short sentence, to solve the user <QUERY>."""

PROMPT_TOT_STATE_EVALUATOR=f"""
For each expert <SOLUTION> evaluate the posibility to reach the solution from user <QUERY> and rank each step solution with confidence score. Consider as most relevant the expert who most closely resembles the area of <QUERY>. Reply with a list of each solution and its confidence score. The confidence score should be between 0 and 100%.
"""
PROMPT_TOT_EXIT_CONDITION= "Review the <EVALUATION> about the user <QUERY>. If the confidence score of the most likely solution is greater than 90%, then reply only with 'STOP'."

user_input = """Tom dejo 3 calcetines en la lavadora, al finalizar, dejo los 3 calcetines secar, luego de 10 horas los calcetines se secaron completamente. ¿Cuánto tiempo llevara secar 20 calcetines?"""

messages = LastMemories()
response = user_input
for i in range(STEPS):
    print(f"----------------------> Step {i+1}:\n")
    # 1.- Descomposición del pensamiento
    messages.clean_memory()
    messages.add_to_memory("system", PROMPT_TOT_GENERATOR)
    if i != 0:
        messages.add_to_memory("user",f"{user_input}")
    solutions = generate_text(response, messages=messages.get_last_memories(n=20))
    print(f"----> SOLUTIONS:\n{solutions}\n")

    # 2.- Evaluación de pensamiento
    messages.clean_memory()
    messages.add_to_memory("system", PROMPT_TOT_STATE_EVALUATOR)
    evaluation = generate_text(f"<QUERY>\n{user_input}\n\n<SOLUTIONS>\n{solutions}", messages=messages.get_last_memories(n=20))
    print(f"----> EVALUATION:\n{evaluation}\n")
    
    # 3.- Condicion de salida
    messages.clean_memory()
    messages.add_to_memory("system", PROMPT_TOT_EXIT_CONDITION)
    response = generate_text(f"<QUERY>\n{user_input}\n\n<EVALUATION>\n{evaluation}", messages=messages.get_last_memories(n=20))
    if response == "STOP":
        print(f"----> EXIT CONDITION:\n{response}\n")
        messages.clean_memory()
        messages.add_to_memory("system", "Based on the user <QUERY> and the <SOLUTIONS> and <EVALUATION>, give a final answer based on the solution with the best confidence score.")
        final_answer = generate_text(f"<QUERY>\n{user_input}\n\n<SOLUTIONS>\n{solutions}\n\n<EVALUATION>\n{evaluation}", messages=messages.get_last_memories(n=20))
        print(final_answer)
        break